In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Iterable, Callable
from utils import load_sequence, visualize, calculate_metrics, visualize_bounding_box

In [ ]:
PEDESTRIAN_PATH: str = r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab2\pedestrian"
OFFICE_PATH: str = r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab2\office"
HIGHWAY_PATH: str = r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab2\highway\highway"

In [ ]:
pedestrian_sequence: List[np.ndarray] = load_sequence(PEDESTRIAN_PATH)
pedestrian_ground_truth: List[np.ndarray] = load_sequence(PEDESTRIAN_PATH, False)
pedestrian_colour : List[np.ndarray] = load_sequence(PEDESTRIAN_PATH, True, False)

office_sequence: List[np.ndarray] = load_sequence(OFFICE_PATH)
office_ground_truth: List[np.ndarray] = load_sequence(OFFICE_PATH, False)
office_colour : List[np.ndarray] = load_sequence(OFFICE_PATH, True, False)

highway_sequence: List[np.ndarray] = load_sequence(HIGHWAY_PATH)
highway_ground_truth: List[np.ndarray] = load_sequence(HIGHWAY_PATH, False)
highway_colour : List[np.ndarray] = load_sequence(HIGHWAY_PATH, True, False)

In [ ]:
def mask(image) -> np.ndarray:
    return cv2.threshold(image, 0, 255, cv2.THRESH_OTSU)[1].astype(np.uint8)*(230>np.max(image)>100).astype(np.uint8)*255

kernel_dilate: np.ndarray = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,23))
kernel_erode: np.ndarray = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,29))
process: Callable[[np.ndarray], np.ndarray] = lambda image: cv2.morphologyEx(cv2.morphologyEx(mask(image), cv2.MORPH_DILATE, kernel_dilate), cv2.MORPH_ERODE, kernel_erode)*255

In [ ]:
def sigma_delta_conservative_enhanced(
        image_stream_grayscale: List[np.ndarray],
        alpha: float = 0.1,
) -> Iterable[np.ndarray]:
    background: np.ndarray = image_stream_grayscale[0].copy().astype(np.float64)
    is_background: np.ndarray = np.zeros_like(background, dtype=np.float64)
    temperature: float = 1.
    for image in image_stream_grayscale:
        background = cv2.multiply((1-alpha) * background + alpha * image.astype(np.float64), is_background*(1-temperature)) + cv2.multiply(background, 1-is_background*(1-temperature))
        diff = cv2.absdiff(image, background.astype(np.uint8))
        is_background = is_background*(temperature) + (1-temperature)*(diff < 10).astype(np.float64)
        temperature -= 0.2
        if temperature < 0.1:
            temperature = 0
        yield diff

In [ ]:
def VIBE(
        image_stream_grayscale: List[np.ndarray], 
        samples_per_pixel: int = 20,
        radius: int = 20,
        min_match_count: int = 2,
        random_subsample: int = 16,
) -> Iterable[np.ndarray]:
    samples = np.zeros((image_stream_grayscale[0].shape[0], image_stream_grayscale[0].shape[1], samples_per_pixel), dtype=np.uint8)
    for image in image_stream_grayscale:
        background = np.zeros_like(image, dtype=np.uint8)
        images = np.stack([image for _ in range(samples_per_pixel)], axis=2)
        dist = np.abs(samples-images)
        count = np.sum(dist < radius, axis=2)
        background = np.where(count >= min_match_count, 0, 255)     
        samples[:, :, np.random.randint(0, samples_per_pixel)] = image
        if np.random.random()* random_subsample < 1: samples[:,:, np.random.randint(0, samples_per_pixel)] = np.roll(image.copy(), np.random.randint(-1, 2), axis=np.random.randint(0, 2))

        yield background

In [ ]:
sg = lambda x: 1 if x else -1

def sgn(x: np.ndarray) -> np.ndarray:
    t = x.copy()
    t[x < 0] = -1
    t[x >= 0] = 1
    return t



def PBAS(
        image_stream_grayscale: List[np.ndarray], 
        samples_per_pixel: int = 35,
        radius: int = 5,
        min_match_count: int = 2,
        r_inc_drc: float = 0.05,
        r_low : int = 18,
        r_scale = 5,
        T_dec = 0.05,
        T_inc = 1.05,
        T_lower = 2,
        T_upper = 200,
) -> Iterable[np.ndarray]:
    samples = np.zeros((image_stream_grayscale[0].shape[0], image_stream_grayscale[0].shape[1], samples_per_pixel), dtype=np.uint8)
    for image in image_stream_grayscale:
        background_model = np.zeros_like(image, dtype=np.uint8)
        samples = np.zeros((image.shape[0], image.shape[1], samples_per_pixel), dtype=np.uint8)
        radius = np.ones_like(image, dtype=np.uint8) * radius
        acc = np.zeros_like(image, dtype=np.uint8)
        D = np.zeros((image.shape[0], image.shape[1], samples_per_pixel) , dtype=np.uint8)
        R = np.ones_like(image, dtype=np.uint8) * r_low
        T = np.ones_like(image, dtype=np.uint8) * T_lower
        background = np.zeros_like(image, dtype=np.uint8)

        images = np.stack([image for _ in range(samples_per_pixel)], axis=2)
        dist = np.abs(samples-images)
        count = np.sum(dist < np.stack([R for _ in range(samples_per_pixel)], axis=2), axis=2)
        background = np.where(count >= min_match_count, 0, 255)

        yield background     


        mask = np.random.uniform(0, 1, size=image.shape) > T

        r = np.random.randint(0, samples_per_pixel)
        rolled = r = np.random.randint(0, samples_per_pixel)
        axis = np.random.randint(0, 2)
        dist = np.random.randint(-1, 2)

        samples[:, :, r] = image*mask + samples[:, :, r]*(1-mask)
        samples[:, :, rolled] = np.roll(image.copy(), dist, axis=axis)*mask + samples[:, :, rolled]*(1-mask)

        r = np.random.randint(0, samples_per_pixel)
        D[:, :, r] = np.min([np.sum(np.abs(image-samples[:,:, i]) for i in range(samples_per_pixel))])*mask + D[:, :, r]*(1-mask)
        acc = np.mean(D, axis=2)
        R = R*(1+sgn(R - r_scale*acc)*r_inc_drc)* mask + R*(1-mask)

        D[:,:, rolled] = np.min([np.sum(np.abs(image-samples[:,:, i]) for i in range(samples_per_pixel))])*mask + D[:,:, rolled]*(1-mask)
        R = R*(1+sgn(R - r_scale*np.mean(D, axis=2))*r_inc_drc)* mask + R*(1-mask)


        T = np.minimum(np.maximum(T_lower, T + sgn(background_model)* T_inc/ (0.01+acc)), T_upper)